In [50]:
import os
import numpy as np
import itertools
import pathos
from pathos.multiprocessing import ProcessingPool as Pool
import matplotlib.pyplot as plt
%matplotlib inline

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, Column, Table, ForeignKey, UniqueConstraint
from sqlalchemy import Integer, String, Float, BLOB
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker

import qp

In [8]:
Base = declarative_base()

In [9]:
def setup(where):
    global parametrizations_table, metaparameters_table, PDFs_table, parameters_table
    global engine, metadata
#     if not os.path.isfile(where):
    engine = create_engine('sqlite:///'+where)
    print('making the tables in '+where)
    metadata = MetaData(engine)
    Base.metadata.create_all(engine)
        # rows are parametrizations keyed on parametrization ID
        # columns are type of parametrization, number of parameters
    parametrizations_table = Table('parametrizations', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('parametrization', String(8), nullable=False),
                    Column('number_metaparameters', Integer, nullable=False)
                    )
        # rows are metaparameters keyed on metaparameter_set ID
        # columns are parametrization ID, metaparameter_set
    metaparameters_table = Table('metaparameters', metadata,
                        Column('id', Integer, primary_key=True),
                        Column('parametrization_id', None, ForeignKey('parametrizations.id')),
                        #Column('metaparameter_number', Integer),          
                        Column('metaparameter_set', BLOB, unique=True)
                        )
        # rows are PDFs keyed on PDF ID
        # no other columns
    PDFs_table = Table('PDFs', metadata,
                  Column('id', Integer, primary_key=True)
                   )
        # rows are parameters keyed on parameter_set ID
        # columns are PDF ID, parametrization ID, parameter_set
    parameters_table = Table('parameters', metadata,
                        Column('id', Integer, primary_key=True),
                         Column('PDFs_id', None, ForeignKey('PDFs.id')),
                         Column('metaparameters_id', None, ForeignKey('metaparameters.id')),
                         #Column('parameter_id', Integer),
                        Column('parameter_set', BLOB),
                        UniqueConstraint('PDFs_id', 'metaparameters_id')#, name=''),
                         )
    metadata.create_all()
#     else:
#         engine = create_engine('sqlite:///'+where)
#         print('loading the tables from '+where)
#         metadata = MetaData(engine)
#         parametrizations_table = Table('parametrizations', metadata, autoload=True)
#         metaparameters_table = Table('metaparameters', metadata, autoload=True)
#         PDFs_table = Table('PDFs', metadata, autoload=True)
#         parameters_table = Table('parameters', metadata, autoload=True)
#         metadata.create_all()

In [10]:
where = 'example.db'
setup(where)

making the tables in example.db


In [11]:
parametrizations = ['samples', 'histogram', 'quantiles']
number_parameters = [3, 10, 30, 100]

for p in parametrizations:
    for n in number_parameters:
        ins = parametrizations_table.insert().values(parametrization=p, number_metaparameters=n)  
        conn = engine.connect()
        result = conn.execute(ins)

In [12]:
s = sqlalchemy.sql.select([parametrizations_table])
result = conn.execute(s).fetchall()
for r in result:
    print(r)

(1, u'samples', 3)
(2, u'samples', 10)
(3, u'samples', 30)
(4, u'samples', 100)
(5, u'histogram', 3)
(6, u'histogram', 10)
(7, u'histogram', 30)
(8, u'histogram', 100)
(9, u'quantiles', 3)
(10, u'quantiles', 10)
(11, u'quantiles', 30)
(12, u'quantiles', 100)


In [14]:
# choose one of these:
dataname = 'mg'
# dataname = 'ss'

if dataname == 'mg':
    # Melissa Graham's data
    datafilename = 'bpz_euclid_test_10_2.probs'
    z_low = 0.01
    z_high = 3.51
elif dataname == 'ss':
    # Sam Schmidt's data
    datafilename = 'test_magscat_trainingfile_probs.out'
    z_low = 0.005
    z_high = 2.11
    
z = np.arange(z_low, z_high, 0.01, dtype='float')
z_range = z_high - z_low
delta_z = z_range / len(z)

## Warning: reading in the data is slow for Sam Schmidt's dataset!
with open(datafilename, 'rb') as data_file:
    lines = (line.split(None) for line in data_file)
    lines.next()
    pdfs = np.array([[float(line[k]) for k in range(1,len(line))] for line in lines])
    
# print(sys.getsizeof(pdfs))

In [16]:
s = sqlalchemy.sql.select([parametrizations_table])
result = conn.execute(s).fetchall()

for r in result:
    if r.parametrization == 'quantiles':
        quantiles = np.linspace(0., 1., r.number_metaparameters).tostring()
        ins = metaparameters_table.insert().prefix_with("OR IGNORE").values(parametrization_id=r.id, metaparameter_set=quantiles)  
        conn = engine.connect()
        result = conn.execute(ins)
    if r.parametrization == 'histogram':
        bindif = 1./r.number_metaparameters
        binends = np.arange(z_low, z_high+bindif, bindif).tostring()
        ins = metaparameters_table.insert().prefix_with("OR IGNORE").values(parametrization_id=r.id, metaparameter_set=binends)  
        conn = engine.connect()
        result = conn.execute(ins)
    if r.parametrization == 'samples':
        n = np.array([r.number_metaparameters]).tostring()
        ins = metaparameters_table.insert().prefix_with("OR IGNORE").values(parametrization_id=r.id, metaparameter_set=n)  
        conn = engine.connect()
        result = conn.execute(ins)

In [17]:
s = sqlalchemy.sql.select([metaparameters_table])
result = conn.execute(s).fetchall()
for r in result:
    print(r)

(1, 1, '\x03\x00\x00\x00\x00\x00\x00\x00')
(2, 2, '\n\x00\x00\x00\x00\x00\x00\x00')
(3, 3, '\x1e\x00\x00\x00\x00\x00\x00\x00')
(4, 4, 'd\x00\x00\x00\x00\x00\x00\x00')
(5, 5, '{\x14\xaeG\xe1z\x84?\xf9\xc5\x92_,\xf9\xd5?\xa7\rt\xda@\xa7\xe5?)\\\x8f\xc2\xf5(\xf0?~\xb1\xe4\x17K~\xf5?\xd3\x06:m\xa0\xd3\xfa?\x14\xaeG\xe1z\x14\x00@\xbeX\xf2\x8b%\xbf\x02@i\x03\x9d6\xd0i\x05@\x14\xaeG\xe1z\x14\x08@\xbeX\xf2\x8b%\xbf\n@i\x03\x9d6\xd0i\r@')
(6, 6, '{\x14\xaeG\xe1z\x84?)\\\x8f\xc2\xf5(\xbc?\xe2z\x14\xaeG\xe1\xca?\xd8\xa3p=\n\xd7\xd3?>\n\xd7\xa3p=\xda?R\xb8\x1e\x85\xebQ\xe0?\x86\xebQ\xb8\x1e\x85\x ... (506 characters truncated) ... @H\xe1z\x14\xaeG\x07@\x14\xaeG\xe1z\x14\x08@\xe1z\x14\xaeG\xe1\x08@\xaeG\xe1z\x14\xae\t@{\x14\xaeG\xe1z\n@H\xe1z\x14\xaeG\x0b@\x14\xaeG\xe1z\x14\x0c@')
(7, 7, '{\x14\xaeG\xe1z\x84?0\x96\xfcb\xc9/\xa6?\xa0\xd3\x06:m\xa0\xb3?)\\\x8f\xc2\xf5(\xbc?Y\xf2\x8b%\xbfX\xc2?\x9d6\xd0i\x03\x9d\xc6?\xe2z\x14\xaeG\xe1\xca ... (2023 characters truncated) ... x14\xaeG\xe1z\n@\xbfX\xf2\x

In [18]:
(n_pdfs, n_start) = np.shape(pdfs)
parametrizations.append('gridded')
number_parameters.append(n_start)

ins = parametrizations_table.insert().values(parametrization=parametrizations[-1], number_metaparameters=number_parameters[-1])  
conn = engine.connect()
result = conn.execute(ins)

s = sqlalchemy.sql.select([parametrizations_table])
result = conn.execute(s).fetchall()
for r in result:
    print(r)

(1, u'samples', 3)
(2, u'samples', 10)
(3, u'samples', 30)
(4, u'samples', 100)
(5, u'histogram', 3)
(6, u'histogram', 10)
(7, u'histogram', 30)
(8, u'histogram', 100)
(9, u'quantiles', 3)
(10, u'quantiles', 10)
(11, u'quantiles', 30)
(12, u'quantiles', 100)
(13, u'gridded', 350)


In [19]:
s = sqlalchemy.sql.select([parametrizations_table]).where(parametrizations_table.c.parametrization=='gridded')
result = conn.execute(s).fetchall()

for r in result:
    ins = metaparameters_table.insert().prefix_with("OR IGNORE").values(parametrization_id=r.id, metaparameter_set=z)  
    conn = engine.connect()
    result = conn.execute(ins)
    
s = sqlalchemy.sql.select([metaparameters_table])
result = conn.execute(s).fetchall()
for r in result:
    print(r)

(1, 1, '\x03\x00\x00\x00\x00\x00\x00\x00')
(2, 2, '\n\x00\x00\x00\x00\x00\x00\x00')
(3, 3, '\x1e\x00\x00\x00\x00\x00\x00\x00')
(4, 4, 'd\x00\x00\x00\x00\x00\x00\x00')
(5, 5, '{\x14\xaeG\xe1z\x84?\xf9\xc5\x92_,\xf9\xd5?\xa7\rt\xda@\xa7\xe5?)\\\x8f\xc2\xf5(\xf0?~\xb1\xe4\x17K~\xf5?\xd3\x06:m\xa0\xd3\xfa?\x14\xaeG\xe1z\x14\x00@\xbeX\xf2\x8b%\xbf\x02@i\x03\x9d6\xd0i\x05@\x14\xaeG\xe1z\x14\x08@\xbeX\xf2\x8b%\xbf\n@i\x03\x9d6\xd0i\r@')
(6, 6, '{\x14\xaeG\xe1z\x84?)\\\x8f\xc2\xf5(\xbc?\xe2z\x14\xaeG\xe1\xca?\xd8\xa3p=\n\xd7\xd3?>\n\xd7\xa3p=\xda?R\xb8\x1e\x85\xebQ\xe0?\x86\xebQ\xb8\x1e\x85\x ... (506 characters truncated) ... @H\xe1z\x14\xaeG\x07@\x14\xaeG\xe1z\x14\x08@\xe1z\x14\xaeG\xe1\x08@\xaeG\xe1z\x14\xae\t@{\x14\xaeG\xe1z\n@H\xe1z\x14\xaeG\x0b@\x14\xaeG\xe1z\x14\x0c@')
(7, 7, '{\x14\xaeG\xe1z\x84?0\x96\xfcb\xc9/\xa6?\xa0\xd3\x06:m\xa0\xb3?)\\\x8f\xc2\xf5(\xbc?Y\xf2\x8b%\xbfX\xc2?\x9d6\xd0i\x03\x9d\xc6?\xe2z\x14\xaeG\xe1\xca ... (2023 characters truncated) ... x14\xaeG\xe1z\n@\xbfX\xf2\x

In [20]:
s = sqlalchemy.sql.select([parametrizations_table.c.id, parametrizations_table.c.number_metaparameters]).where(parametrizations_table.c.parametrization=='gridded')
latest = conn.execute(s).fetchall()
print(type(latest))

<type 'list'>


In [21]:
s1 = sqlalchemy.sql.select([parametrizations_table.c.id]).where(parametrizations_table.c.parametrization=='gridded' and parametrizations_table.c.number_metaparameters==n_start)
metaparam_index = conn.execute(s1).fetchall()[0]
print(metaparam_index)

s2 = sqlalchemy.sql.select([metaparameters_table.c.metaparameter_set]).where(metaparameters_table.c.parametrization_id==metaparam_index[0])
metaparam_bytes = conn.execute(s2).fetchall()
print(metaparam_bytes)
in_metaparams = np.fromstring(metaparam_bytes[0][0])
print(in_metaparams)

(13,)
[('{\x14\xaeG\xe1z\x84?{\x14\xaeG\xe1z\x94?\xb8\x1e\x85\xebQ\xb8\x9e?{\x14\xaeG\xe1z\xa4?\x9a\x99\x99\x99\x99\x99\xa9?\xb9\x1e\x85\xebQ\xb8\xae?\xebQ\xb ... (7469 characters truncated) ... 9\x99\x99\x99\x99\x99\x0b@\xaeG\xe1z\x14\xae\x0b@\xc2\xf5(\\\x8f\xc2\x0b@\xd7\xa3p=\n\xd7\x0b@\xebQ\xb8\x1e\x85\xeb\x0b@\x00\x00\x00\x00\x00\x00\x0c@',)]
[ 0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12
  0.13  0.14  0.15  0.16  0.17  0.18  0.19  0.2   0.21  0.22  0.23  0.24
  0.25  0.26  0.27  0.28  0.29  0.3   0.31  0.32  0.33  0.34  0.35  0.36
  0.37  0.38  0.39  0.4   0.41  0.42  0.43  0.44  0.45  0.46  0.47  0.48
  0.49  0.5   0.51  0.52  0.53  0.54  0.55  0.56  0.57  0.58  0.59  0.6
  0.61  0.62  0.63  0.64  0.65  0.66  0.67  0.68  0.69  0.7   0.71  0.72
  0.73  0.74  0.75  0.76  0.77  0.78  0.79  0.8   0.81  0.82  0.83  0.84
  0.85  0.86  0.87  0.88  0.89  0.9   0.91  0.92  0.93  0.94  0.95  0.96
  0.97  0.98  0.99  1.    1.01  1.02  1.03  1.04  1.05  1.06  1.07  

In [22]:
ins = PDFs_table.insert()  
conn = engine.connect()
result = conn.execute(ins, [{}]*n_pdfs)

s = sqlalchemy.sql.select([PDFs_table.c.id])
result = conn.execute(s).fetchall()
for r in result[:10]:
    print(r[0])

1
2
3
4
5
6
7
8
9
10


In [23]:
s1 = sqlalchemy.sql.select([PDFs_table.c.id])
pdf_indices = np.array(conn.execute(s1).fetchall()[:100])
print(type(pdf_indices), type(pdf_indices[0]), type(pdf_indices[0][0]))
s2 = sqlalchemy.sql.select([metaparameters_table.c.id]).where(metaparameters_table.c.metaparameter_set==in_metaparams.tostring())
metaparam_id = conn.execute(s2).fetchall()[0][0]
print(metaparam_id)

def make_parameter_dicts(i):
    new_parameters = {'PDFs_id' : i[0], 'metaparameters_id' : metaparam_id, 'parameter_set' : pdfs[i[0]-1].tostring()}
    return new_parameters
    
pool = Pool(2)
parameter_dicts = pool.map(make_parameter_dicts, pdf_indices)
print(len(parameter_dicts))

# def pdfs_from_file_to_db(i):
# #     P = qp.PDF(gridded=(in_metaparams, pdfs[i]), vb=False)
#     ins = parameters_table.insert().values(PDFs_id=i[0], metaparameters_id=metaparam_id, parameter_set=pdfs[i].tostring())  
#     conn = engine.connect()
#     result = conn.execute(ins)

# pool = Pool(2)
# pool.map(pdfs_from_file_to_db, pdf_indices : 

(<type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.int64'>)
13
100


In [24]:
conn = engine.connect()
conn.execute(parameters_table.insert(), parameter_dicts)

# s = sqlalchemy.sql.select([parameters_table.c.id])
# result = conn.execute(s).fetchall()
# for r in result[:10]:
#     print(r[0])

In [43]:
sxi = sqlalchemy.sql.select([metaparameters_table.c.metaparameter_set]).where(metaparameters_table.c.id==metaparam_id)
xi = np.fromstring(conn.execute(sxi).fetchall()[0][0])

# sy = sqlalchemy.sql.select([parameters_table.c.parameter_set]).where(parameters_table.c.metaparameters_id==metaparam_id)
# y = conn.execute(sy).fetchall()

sp = sqlalchemy.sql.select([parametrizations_table.c.id]).where(parametrizations_table.c.parametrization=='samples')
ps = np.array(conn.execute(sp).fetchall())
# for p in ps:
#     print p

sf = sqlalchemy.sql.select([metaparameters_table.c.id]).where(metaparameters_table.c.parametrization_id.in_(ps[:,0]))
fs = np.array(conn.execute(sf).fetchall())[:,0]
# for f in fs:
#     print f[0]
print(fs)

[1 2 3 4]


In [52]:
def gridded_to_samples((i, f)):
    print(i, f)
    conn = engine.connect()
    sxi = sqlalchemy.sql.select([metaparameters_table.c.metaparameter_set]).where(metaparameters_table.c.id==metaparam_id)
    xi = np.fromstring(conn.execute(sxi).fetchall()[0][0])
    syi = sqlalchemy.sql.select([parameters_table.c.parameter_set]).where(parameters_table.c.metaparameters_id==metaparam_id and parameters_table.c.PDFs_id==i[0])
    yi = np.fromstring(conn.execute(syi).fetchall()[0][0])
    G = qp.PDF(gridded=(xi, yi))
    sxf = sqlalchemy.sql.select([metaparameters_table.c.metaparameter_set]).where(metaparameters_table.c.id==f)
    xf = np.fromstring(conn.execute(sxf).fetchall()[0][0], dtype=int)
    S = G.sample(N=xf, using='gridded')
    ins = parameters_table.insert().prefix_with("OR IGNORE").values(PDFs_id=i[0], metaparameters_id=f, parameter_set=S.tostring())  
    result = conn.execute(ins)
    return S

In [53]:
conn = engine.connect()
new_pdfs = pool.map(gridded_to_samples, itertools.product(pdf_indices[:100], fs))

In [54]:
s = sqlalchemy.sql.select([parameters_table.c.parameter_set]).where(parameters_table.c.PDFs_id==5)
result = conn.execute(s).fetchall()
for r in result:
    print np.fromstring(r[0])

[ 0.66867775  0.7138382   0.64518954]
[ 0.71852174  0.71004979  0.73212906  0.74416141  0.71642063  0.68372763
  0.68642698  0.69862394  0.73726591  0.70540124]
[ 0.69798101  0.73117395  0.69263285  0.68966414  0.71044281  0.73829885
  0.70313301  0.6628309   0.70279409  0.66158559  0.67158146  0.66408125
  0.70739304  0.68323288  0.71007485  0.70990671  0.7271088   0.70970038
  0.6620399   0.68422075  0.6650514   0.72758056  0.6631331   0.6999295
  0.71335944  0.6696978   0.72905384  0.68247447  0.66620077  0.69392012]
[ 0.7069089   0.71184138  0.70071943  0.70753279  0.71146038  0.71647714
  0.67890381  0.70846752  0.72449606  0.69060449  0.70658745  0.70780204
  0.70399413  0.6835401   0.68608515  0.67440739  0.71612201  0.72077507
  0.71860996  0.71752293  0.67652585  0.70229307  0.70096997  0.718839
  0.69569901  0.73356887  0.73349896  0.68685665  0.66817465  0.71340672
  0.67985999  0.70754323  0.6572879   0.71148023  0.68276459  0.66388681
  0.71210975  0.67969109  0.70021317  

In [20]:
s = sqlalchemy.sql.select([parameters_table.c.id])
parameter_indices = conn.execute(s).fetchall()
for p in parameter_indices:
    print(p)

(1,)


In [ ]:
for p in range(n_pdfs):
    ins = parametrizations_table.insert().values(parameters=pdfs[p].tostring(), number_metaparameters=n)  
    conn = engine.connect()
    result = conn.execute(ins)

In [7]:

s = sqlalchemy.sql.select([parametrizations_table]).where(parametrizations_table.c.parametrization=='quantiles')
result = conn.execute(s)

out_place = result.fetchall() 
for o in out_place:
    quantiles = np.linspace(0., 1., r.number_metaparameters).tostring()
    ins = metaparameters_table.insert().values(parametrization_id=o.id, metaparameter_set=quantiles)  
    conn = engine.connect()
    result = conn.execute(ins)
    

9
        [�R�֯�?[�R�֯�?|���?[�R�֯�?2ogH�۩?|���?pgH���?[�R�֯�?F]t�E�?2ogH�۹?�q�q�?|���?z;Cb���?pgH���?e�M6�d�?[�R�֯�?Q�W
���?F]t�E�?<Cb�ΐ�?2ogH���?'�l��&�?�q�q�?�v�ļ�?|���?�@�_)�?z;Cb���?u�E]t�?pgH���?j�J�Z��?e�M6�d�?`)P�W
�?[�R�֯�?VUUUUU�?Q�W
���?L�Z�R��?F]t�E�?A�_)P��?<Cb�ΐ�?7�d�M6�?2ogH���?-j�J��?'�l��&�?"1ogH��?�q�q�?]t�E�?�v�ļ�?�y;Cb�?|���?�~�@��?�@�_)�?}��|�?z;Cb���?w�ļ�!�?u�E]t�?r�q��?pgH���?m��&�l�?j�J�Z��?hH���?e�M6�d�?c�ΐ���?`)P�W
�?^t�E]�?[�R�֯�?X
����?VUUUUU�?S�֯��?Q�W
���?N6�d�M�?L�Z�R��?I����?F]t�E�?Db�ΐ��?A�_)P��??��>�?<Cb�ΐ�?9��8���?7�d�M6�?4$����?2ogH���?/�袋.�?-j�J��?*P�W
��?'�l��&�?%���y�?"1ogH��? |���?�q�q�?�v���?]t�E�?��+j�?�v�ļ�?>���?�y;Cb�?�����?|���?j�J�Z�?�~�@��?      �?
10
        [�R�֯�?[�R�֯�?|���?[�R�֯�?2ogH�۩?|���?pgH���?[�R�֯�?F]t�E�?2ogH�۹?�q�q�?|���?z;Cb���?pgH���?e�M6�d�?[�R�֯�?Q�W
���?F]t�E�?<Cb�ΐ�?2ogH���?'�l��&�?�q�q�?�v�ļ�?|���?�@�_)�?z;Cb���?u�E]t�?pg

In [8]:
#print(np.fromiter(metaparameters_table.fetchall(), np.ndarray).fromstring())
s = sqlalchemy.sql.select([metaparameters_table])
result = conn.execute(s)

for r in result.fetchall():
    print(r)

(1, 9, '\x00\x00\x00\x00\x00\x00\x00\x00[\xbfR\xa0\xd6\xaf\x84?[\xbfR\xa0\xd6\xaf\x94?\x08\x1f|\xf0\xc1\x07\x9f?[\xbfR\xa0\xd6\xaf\xa4?2ogH\xcc\xdb\xa9?\x08\ ... (1784 characters truncated) ... \xee?\x0e\x89y;Cb\xee?\x0b\xd4\xfa\x95\x02\xb5\xee?\x08\x1f|\xf0\xc1\x07\xef?\x06j\xfdJ\x81Z\xef?\x03\xb5~\xa5@\xad\xef?\x00\x00\x00\x00\x00\x00\xf0?')
(2, 10, '\x00\x00\x00\x00\x00\x00\x00\x00[\xbfR\xa0\xd6\xaf\x84?[\xbfR\xa0\xd6\xaf\x94?\x08\x1f|\xf0\xc1\x07\x9f?[\xbfR\xa0\xd6\xaf\xa4?2ogH\xcc\xdb\xa9?\x08\ ... (1784 characters truncated) ... \xee?\x0e\x89y;Cb\xee?\x0b\xd4\xfa\x95\x02\xb5\xee?\x08\x1f|\xf0\xc1\x07\xef?\x06j\xfdJ\x81Z\xef?\x03\xb5~\xa5@\xad\xef?\x00\x00\x00\x00\x00\x00\xf0?')
(3, 11, '\x00\x00\x00\x00\x00\x00\x00\x00[\xbfR\xa0\xd6\xaf\x84?[\xbfR\xa0\xd6\xaf\x94?\x08\x1f|\xf0\xc1\x07\x9f?[\xbfR\xa0\xd6\xaf\xa4?2ogH\xcc\xdb\xa9?\x08\ ... (1784 characters truncated) ... \xee?\x0e\x89y;Cb\xee?\x0b\xd4\xfa\x95\x02\xb5\xee?\x08\x1f|\xf0\xc1\x07\xef?\x06j\xfdJ\x81Z\xef?\x03\xb5~\xa5@\

In [ ]:
for q in result:
    quantiles = np.linspace(re)
    ins = metaparameters_table.insert().values(parametrization=p, number_metaparameters=n) 
    conn = engine.connect()
    result = conn.execute(ins)
    print(r)

In [ ]:
for r in result:
    print(r.id)

In [ ]:
ps = sqlalchemy.sql.select([parametrizations_table])
result = conn.execute(ps)
for p in result:
    print p

In [ ]:
import dataset as ds
# help(dataset)

In [ ]:
db = ds.connect('postgresql:///example.db')

In [ ]:
import peewee as pw
help(pw)

from peewee import Model, CharField

In [ ]:
import playhouse as ph
help(ph)
from playhouse.postgres_ext import PostgresqlExtDatabase

In [ ]:
psql_db = PostgresqlExtDatabase('example', user='postgres')

class BaseModel(Model):
    class Meta:
        database = psql_db

# class Galaxy(BaseModel):
#     Galaxy_id = pw.PrimaryKey()
    
# class PDF(BaseModel):
#     PDF_id = pw.ForeignKeyField(Galaxy)
#     parametrization_id = pw.ForeignKeyField(Parametrization)
#     parameters = Array()
    
class Parametrization(BaseModel):
    P_type = CharField()
    parameters = ph.postgres_ext.ArrayField(pw.FloatField)

In [ ]:
psql_db.connect()

In [ ]:
def setup(where):
    global parametrizations_table, metaparameters_table, PDFs_table, parameters_table
    global engine, metadata
    if not os.path.isfile(where):
        engine = create_engine('sqlite:///'+where)
        print('making the tables in '+where)
        metadata = MetaData(engine)
        Base.metadata.create_all(engine)
        parametrizations_table = Table('parametrizations', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('parametrization', String(8))
                    )
 
        metaparameters_table = Table('metaparameters', metadata,
                        Column('id', Integer, primary_key=True),
                        Column('parametrization_id', None, ForeignKey('parametrizations.id')),
                        Column('metaparameter', ARRAY(Float))                            
                        )

        PDFs_table = Table('PDFs', metadata,
                  Column('id', Integer, primary_key=True)
                   )
        parameters_table = Table('parameters', metadata,
                        Column('id', Integer, primary_key=True),
                         Column('PDFs_id', None, ForeignKey('PDFs.id')),
                         Column('metaparameters_id', None, ForeignKey('metaparameters.id')),
                         Column('parameter', ARRAY(Float)) 
                         )
        metadata.create_all()
    else:
        engine = create_engine('sqlite:///'+where)
        print('loading the tables from '+where)
        metadata = MetaData(engine)
        parametrizations_table = Table('parametrizations', metadata, autoload=True)
        metaparameters_table = Table('metaparameters', metadata, autoload=True)
        PDFs_table = Table('PDFs', metadata, autoload=True)
        parameters_table = Table('parameters', metadata, autoload=True)
        metadata.create_all()

In [ ]:
from sqlalchemy.dialects import postgresql
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Column, Table, ForeignKey
from sqlalchemy import Integer, String, ARRAY
 
engine = create_engine('postgresql:///home/aimalz/Code/qp/docs/desc-0000-qp-photo-z_approximation/research/ensemble.db',
                       echo=True)
 
metadata = MetaData(bind=engine)
 
parametrizations_table = Table('parametrizations', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('parametrization', String(8)),
                    )
 
metaparameters_table = Table('metaparameters', metadata,
                        Column('id', Integer, primary_key=True),
                        Column('parametrization_id', None, ForeignKey('parametrizations.id')),
                        Column('metaparameters', ARRAY(float), nullable=False)                            
                        )
 
# create tables in database
metadata.create_all()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
test = pd.DataFrame()
print(test)

In [ ]:
print(test.columns)
# print(full_df.columns)

In [ ]:
df_columns = ['quantiles', 'histogram', 'samples', 'gridded', 'gmm']
test = pd.DataFrame(columns = df_columns)
print(type(test.index))

In [ ]:
df_columns = ['quantiles', 'histogram', 'samples', 'gridded', 'gmm']
data1 = {'quantiles': np.arange(10), 'histogram': np.arange(10)}
test_df = pd.DataFrame([data1], index = ['first'])
print(test_df.index)

In [ ]:
data2 = {'samples': np.arange(10), 'histogram': np.arange(10)}

another_df = pd.DataFrame([data2], index = ['second'])
combo_df = test_df.append(another_df)
print(combo_df.index)

In [ ]:
galinds = np.arange(0, 1, 0.1)
galnames = [str(ind) for ind in galinds]
print(galnames)

empty_df = pd.DataFrame(None)#, index = galnames)
print(empty_df)

In [ ]:
new_galinds = galinds + 0.5
new_galnames = [str(ind) for ind in new_galinds]
old_df = pd.DataFrame([data1] * 10, index = galnames)
#for galname in galnames:
new_df = pd.DataFrame([data2] * 10, index = new_galnames)
# full_df = pd.concat([old_df, new_df], axis=1)
# print(full_df)

In [ ]:
# data1pp, data2pp = [], []
# for i in range(len(galnames)):
#     print(galnames[i])
#     td1 = data1
#     td1['galname'] = galnames[i]
#     data1pp.append(td1)
#     td2 = data2
#     td2['galname'] = galnames[i]
#     data2pp.append(td2)
# full_df = pd.DataFrame(data1pp)
# print(full_df)
# new_df = pd.DataFrame(data2pp)#, index = galnames)
# print(new_df)
#full_df = pd.merge(old_df, new_df, how = 'outer', on = 'index')
full_df = old_df.combine_first(new_df)
print(full_df)

The `qp.Ensemble` class should relate a few structures:
* Something containing identifiers (immutable) and qp.PDF objects (mutable)
* A database of columns of parametrizations and rows of parameters (or None)

In [ ]:
#help(parametrizations_table)
print(parametrizations_table.metadata)

In [ ]:
metadata = MetaData()

Ensemble = Table('ensemble', metadata,
    Column('meta_mixmod', ARRAY(String)),
    Column('meta_quantiles', ARRAY(Float)),
    Column('meta_histogram', ARRAY(Float)),
    Column('meta_gridded', ARRAY(Float)),
    Column('meta_samples', ARRAY(Float)))

In [ ]:
class Galaxy(Base):
    __tablename__ = 'galaxy'
    idno = Column(Integer, primary_key=True)
    pdf = Column(BLOB)

In [ ]:
class Parametrizations(Base):
    __tablename__ = 'parametrizations'
    idno = Column(Integer, primary_key=True)
    mixmod = Column(ARRAY(Float))
    quantiles = Column(ARRAY(Float))
    histogram = Column(ARRAY(Float))
    gridded = Column(ARRAY(Float))
    samples = Column(ARRAY(Float))
    galaxy = Relationship(Galaxy)

In [ ]:
testpdf = qp.PDF()

In [ ]:
help(sqlalchemy)